In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch

In [ ]:
df = pd.read_csv("/content/cleaned_kaggle_data.csv")

In [ ]:
df.head()

,Unnamed: 0,Topics,Date,bias_label,text,bias_prefix
0,0,Elections,2012-07-26,left_story_text,Mitt Romney told British leaders he hopes to t...,left
1,1,Elections,2012-08-01,left_story_text,OPINION\nWarsaw (CNN) -- In the estimation of ...,left
2,2,Elections,2012-08-16,left_story_text,The most emotionally powerful minute of Joe Bi...,left
3,3,Elections,2012-08-17,left_story_text,With the spotlight back on Mitt Romney's tax r...,left
4,4,US Congress,2012-08-20,left_story_text,The FBI probed a late-night swim in the Sea of...,left


In [ ]:
df = df.drop(columns = ['Unnamed: 0', 'bias_label'])

In [ ]:
df = df[df['text'].str.len()>150]

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(model_name)

In [ ]:
# class SentenceTransformerWrapper(nn.Module):
#     def __init__(self, model_name):
#         super(SentenceTransformerWrapper, self).__init__()
#         self.linear = nn.Linear(768, 3)
#         self.model = SentenceTransformer(model_name)

#     def forward(self, x):
#         embeddings = self.model.forward(x, convert_to_tensor=True)
#         return torch.softmax(self.linear(embeddings))
# #Calulating loss and then the derivative and backpropogates it
# #Optimizer.step
# thing = SentenceTransformerWrapper()
# Optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# SentenceTransformerWrapper(model_name)
# SentenceTransformerWrapper.forward(df["text"].tolist())
# loss = torch.cross_entropy(output, target)
# loss.backward()
Optimizer.step()
# Iterate through all the points in your data for stochastic gradient descent
# Look online how to freeze the first x layers

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel

class SentenceTransformerWrapper(nn.Module):
    def __init__(self, model_name='roberta-base'):
        super().__init__()
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.roberta = RobertaModel.from_pretrained(model_name)
        self.classifier = nn.Linear(768, num_classes)

    def forward(self, x):
        # embeddings = self.model.encode(x, convert_to_tensor=True)
        embeddings = self.model.encode(x, convert_to_tensor=True)
        logits = self.linear(embeddings)
        return logits   # DO NOT softmax here!


In [ ]:
model_name = "sentence-transformers/all-roberta-large-v1"
wrapper = SentenceTransformerWrapper(model_name)

optimizer = torch.optim.Adam(wrapper.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

texts = df["text"].tolist()
targets = torch.tensor(df["bias"].tolist())

for epoch in range(5):
    logits = wrapper(texts)
    loss = criterion(logits, targets)

    # accuracy
    preds = torch.argmax(logits, dim=1)
    acc = (preds == targets).float().mean().item()

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f} | Acc: {acc:.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
embeddings = model.encode(
    df['text'].tolist(),
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

Batches:   0%|          | 0/1456 [00:00<?, ?it/s]

In [ ]:
print("Embedding matrix shape:", embeddings.shape)


In [ ]:
np.save("text_embeddings.npy", embeddings)
df["embedding_index"] = range(len(df))  # optional mapping for analysis
df.to_csv("articles_with_indices.csv", index=False)

In [ ]:
emb = np.load("text_embeddings.npy")
df = pd.read_csv("articles_with_indices.csv")

labels = df["bias_prefix"]

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["dayofweek"] = df["Date"].dt.dayofweek

# Topic one-hot encoding
topic_dummies = pd.get_dummies(df["Topics"], prefix="topic")

# Combine everything
X = np.hstack([topic_dummies.values,
               df[["year","month","dayofweek"]].values])

y = labels

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=2000, solver="liblinear")
clf.fit(X_train, y_train)

LogisticRegression(max_iter=2000, solver='liblinear')

In [ ]:
y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro F1:", f1_score(y_test, y_pred, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred, average="weighted"))

Accuracy: 0.3455677183945053
Macro F1: 0.22161085438581707
Weighted F1: 0.22970191756225455


In [ ]:
bias_map = {
    "left": 0,
    "center": 1,
    "right": 2
}

df["bias"] = df["bias_prefix"].map(bias_map)
df.to_csv("articles_with_indices.csv", index=False)

In [ ]:
emb = np.load("text_embeddings.npy")
df = pd.read_csv("articles_with_indices.csv")
y = df["bias"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    emb, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    max_iter=5000,
    solver="saga",
    n_jobs=-1
)

clf.fit(X_train, y_train)

LogisticRegression(max_iter=5000, n_jobs=-1, solver='saga')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro F1:", f1_score(y_test, y_pred, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred, average="weighted"))
print(classification_report(y_test, y_pred))

Accuracy: 0.40223223867782787
Macro F1: 0.39681434419239303
Weighted F1: 0.3989574779420947
              precision    recall  f1-score   support

           0       0.42      0.42      0.42      1594
           1       0.38      0.30      0.33      1450
           2       0.40      0.47      0.44      1615

    accuracy                           0.40      4659
   macro avg       0.40      0.40      0.40      4659
weighted avg       0.40      0.40      0.40      4659



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(df["text"])

from scipy.sparse import hstack
X_full = hstack([X_tfidf, emb])


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_full,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# -----------------------------
# Logistic Regression Classifier
# -----------------------------
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    max_iter=5000,
    solver="saga",
    n_jobs=-1
)

clf.fit(X_train, y_train)

# -----------------------------
# Evaluation
# -----------------------------
from sklearn.metrics import accuracy_score, f1_score, classification_report

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro F1:", f1_score(y_test, y_pred, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred, average="weighted"))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Accuracy: 0.4565357372826787
Macro F1: 0.4524212300636686
Weighted F1: 0.45449134357940857

Classification Report:

              precision    recall  f1-score   support

           0       0.46      0.46      0.46      1594
           1       0.42      0.37      0.39      1450
           2       0.48      0.53      0.50      1615

    accuracy                           0.46      4659
   macro avg       0.45      0.45      0.45      4659
weighted avg       0.45      0.46      0.45      4659



In [ ]:
df.drop(columns=['bias_prefix', 'Date'], inplace=True)

In [ ]:
df.head()

,Topics,text,embedding_index,year,month,dayofweek,bias
0,Elections,Mitt Romney told British leaders he hopes to t...,0,2012,7,3,0
1,Elections,OPINION\nWarsaw (CNN) -- In the estimation of ...,1,2012,8,2,0
2,Elections,With the spotlight back on Mitt Romney's tax r...,2,2012,8,4,0
3,US Congress,The FBI probed a late-night swim in the Sea of...,3,2012,8,0,0
4,Elections,Representative Todd Akin on Tuesday ignored a ...,4,2012,8,2,0
